# Fast Style Transfer for Arbitrary Styles


Based on the model code in [magenta](https://github.com/tensorflow/magenta/tree/master/magenta/models/arbitrary_image_stylization) and the publication:

[Exploring the structure of a real-time, arbitrary neural artistic stylization
network](https://arxiv.org/abs/1705.06830).
*Golnaz Ghiasi, Honglak Lee,
Manjunath Kudlur, Vincent Dumoulin, Jonathon Shlens*,
Proceedings of the British Machine Vision Conference (BMVC), 2017.

Also check the [Intuitive Guide to Neural Style Transfer](https://towardsdatascience.com/light-on-math-machine-learning-intuitive-guide-to-neural-style-transfer-ef88e46697ee) in Towards Data Science by Thushan Ganegedara.

## Setup

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Let's start with importing TF2 and all relevant dependencies.

In [ ]:
import functools
import os

from matplotlib import gridspec
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

print("TF Version: ", tf.__version__)
print("TF Hub version: ", hub.__version__)
print("Eager mode enabled: ", tf.executing_eagerly())
print("GPU available: ", tf.config.list_physical_devices('GPU'))

## Create some helper functions

In [ ]:
# @title Crop the image as a square, starting from its center
def crop_center(image):
  """Returns a cropped square image."""
  shape = image.shape
  new_shape = min(shape[1], shape[2])
  offset_y = max(shape[1] - shape[2], 0) // 2
  offset_x = max(shape[2] - shape[1], 0) // 2
  image = tf.image.crop_to_bounding_box(
      image, offset_y, offset_x, new_shape, new_shape)
  return image

In [ ]:
# @title Load the image into a TensorFlow/Keras object
@functools.lru_cache(maxsize=None)
def load_image(image_url, image_size=(256, 256), preserve_aspect_ratio=True):
  """Loads and preprocesses images."""
  # Cache image file locally.
  image_path = tf.keras.utils.get_file(os.path.basename(image_url)[-128:], image_url)
  # Load and convert to float32 numpy array, add batch dimension, and normalize to range [0, 1].
  img = tf.io.decode_image(
      tf.io.read_file(image_path),
      channels=3, dtype=tf.float32)[tf.newaxis, ...]
  img = crop_center(img)
  img = tf.image.resize(img, image_size, preserve_aspect_ratio=True)
  return img

In [ ]:
# @title Show several images side by side 
def show_n(images, titles=('',)):
  n = len(images)
  image_sizes = [image.shape[1] for image in images]
  w = (image_sizes[0] * 6) // 320
  plt.figure(figsize=(w * n, w))
  gs = gridspec.GridSpec(1, n, width_ratios=image_sizes)
  for i in range(n):
    plt.subplot(gs[i])
    plt.imshow(images[i][0], aspect='equal')
    plt.axis('off')
    plt.title(titles[i] if len(titles) > i else '')
  plt.show()


## Import images

In [ ]:
#@title Import a series of content and style images { display-mode: "form" }
content_urls = dict(
  ist1='https://www.innoenergy.com/media/1626/ist_resized.jpg',
  ist2='https://www.ulisboa.pt/sites/ulisboa.pt/files/uo/images/ist_exterior7_copy.jpg',
  ist3='https://www.ipfn.tecnico.ulisboa.pt/APPLAuSE/img/uptecnico.jpg',
  lisbon='https://ak.picdn.net/shutterstock/videos/34805464/thumb/1.jpg?ip=x480'
  )
style_urls = dict(
  style1='https://github.com/thushv89/exercises_thushv_dot_com/blob/master/neural_style_transfer_light_on_math_ml/data/style_1.jpg?raw=true',
  style2='https://github.com/thushv89/exercises_thushv_dot_com/blob/master/neural_style_transfer_light_on_math_ml/data/style_3.jpg?raw=true',
  style3='https://github.com/thushv89/exercises_thushv_dot_com/blob/master/neural_style_transfer_light_on_math_ml/data/style_6.jpg?raw=true',
  style4='https://github.com/thushv89/exercises_thushv_dot_com/blob/master/neural_style_transfer_light_on_math_ml/data/style_7.jpg?raw=true',
  style5='https://github.com/thushv89/exercises_thushv_dot_com/blob/master/neural_style_transfer_light_on_math_ml/data/style_8.jpg?raw=true',
  style6='https://github.com/thushv89/exercises_thushv_dot_com/blob/master/neural_style_transfer_light_on_math_ml/data/style_9.jpg?raw=true',
  style7='https://github.com/thushv89/exercises_thushv_dot_com/blob/master/neural_style_transfer_light_on_math_ml/data/style_10.jpg?raw=true',
  kanagawa_great_wave='https://upload.wikimedia.org/wikipedia/commons/0/0a/The_Great_Wave_off_Kanagawa.jpg',
  kandinsky_composition_7='https://upload.wikimedia.org/wikipedia/commons/b/b4/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg',
  hubble_pillars_of_creation='https://upload.wikimedia.org/wikipedia/commons/6/68/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg',
  van_gogh_starry_night='https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg',
  turner_nantes='https://upload.wikimedia.org/wikipedia/commons/b/b7/JMW_Turner_-_Nantes_from_the_Ile_Feydeau.jpg',
  munch_scream='https://upload.wikimedia.org/wikipedia/commons/c/c5/Edvard_Munch%2C_1893%2C_The_Scream%2C_oil%2C_tempera_and_pastel_on_cardboard%2C_91_x_73_cm%2C_National_Gallery_of_Norway.jpg',
  picasso_demoiselles_avignon='https://upload.wikimedia.org/wikipedia/en/4/4c/Les_Demoiselles_d%27Avignon.jpg',
  picasso_violin='https://upload.wikimedia.org/wikipedia/en/3/3c/Pablo_Picasso%2C_1911-12%2C_Violon_%28Violin%29%2C_oil_on_canvas%2C_Kr%C3%B6ller-M%C3%BCller_Museum%2C_Otterlo%2C_Netherlands.jpg',
  picasso_bottle_of_rum='https://upload.wikimedia.org/wikipedia/en/7/7f/Pablo_Picasso%2C_1911%2C_Still_Life_with_a_Bottle_of_Rum%2C_oil_on_canvas%2C_61.3_x_50.5_cm%2C_Metropolitan_Museum_of_Art%2C_New_York.jpg',
  fire='https://upload.wikimedia.org/wikipedia/commons/3/36/Large_bonfire.jpg',
  derkovits_woman_head='https://upload.wikimedia.org/wikipedia/commons/0/0d/Derkovits_Gyula_Woman_head_1922.jpg',
  amadeo_style_life='https://upload.wikimedia.org/wikipedia/commons/8/8e/Untitled_%28Still_life%29_%281913%29_-_Amadeo_Souza-Cardoso_%281887-1918%29_%2817385824283%29.jpg',
  derkovtis_talig='https://upload.wikimedia.org/wikipedia/commons/3/37/Derkovits_Gyula_Talig%C3%A1s_1920.jpg',
  amadeo_cardoso='https://upload.wikimedia.org/wikipedia/commons/7/7d/Amadeo_de_Souza-Cardoso%2C_1915_-_Landscape_with_black_figure.jpg'
)

content_image_size = 384
style_image_size = 256
content_images = {k: load_image(v, (content_image_size, content_image_size)) for k, v in content_urls.items()}
style_images = {k: load_image(v, (style_image_size, style_image_size)) for k, v in style_urls.items()}
style_images = {k: tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME') for k, style_image in style_images.items()}


## Import TF Hub module

Style tranfer models are Deep Learning neural networks used for this specific purpose. Generally, they have to be manually trained. However, for this tutorial, we can load pre-trained models from TensorFlow Hub, a repository of trained machine learning models. 

In [ ]:
# Load TF Hub module.

hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
hub_module = hub.load(hub_handle)

The signature of this hub module for image stylization is:
```
outputs = hub_module(content_image, style_image)
stylized_image = outputs[0]
```
Where `content_image`, `style_image`, and `stylized_image` are expected to be 4-D Tensors with shapes `[batch_size, image_height, image_width, 3]`.

In the current example we provide only single images and therefore the batch dimension is 1, but one can use the same module to process more images at the same time.

The input and output values of the images should be in the range [0, 1].

The shapes of content and style image don't have to match. Output image shape
is the same as the content image shape.

In [ ]:
#@title Specify the main content image and the style you want to use.  { display-mode: "form" }

content_name = 'lisbon'  # @param ['ist1', 'ist2', 'ist3', 'lisbon']
style_name = 'van_gogh_starry_night'  # @param ['style1','style2','style3','style4','style5','style6','style7','kanagawa_great_wave', 'kandinsky_composition_7', 'hubble_pillars_of_creation', 'van_gogh_starry_night', 'turner_nantes', 'munch_scream', 'picasso_demoiselles_avignon', 'picasso_violin', 'picasso_bottle_of_rum', 'fire', 'derkovits_woman_head', 'amadeo_style_life', 'derkovtis_talig', 'amadeo_cardoso']

stylized_image = hub_module(tf.constant(content_images[content_name]),
                            tf.constant(style_images[style_name]))[0]

show_n([content_images[content_name], style_images[style_name], stylized_image],
       titles=['Original content image', 'Style image', 'Stylized image'])

## Compare GPU vs CPU performance

We can confirm that it is possible to connect to the GPU with tensorflow by running the following code:

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

If the GPU is found, we can run a style transfer example, comparing the performance of using the CPU vs using the GPU.

In [ ]:
#%tensorflow_version 2.x
#import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
content_id = 'ist3'
style_id = 'munch_scream'

def cpu():
  with tf.device('/cpu:0'):
    stylized_image_cpu = hub_module(tf.constant(content_images[content_id]),
                            tf.constant(style_images[style_id]))[0]
    return stylized_image_cpu

def gpu():
  with tf.device('/device:GPU:0'):
    stylized_image_gpu = hub_module(tf.constant(content_images[content_id]),
                            tf.constant(style_images[style_id]))[0]
    return stylized_image_gpu
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
stylized_image_cpu = cpu()
stylized_image_gpu = gpu()

# Run the op several times.
cpu_time = timeit.timeit('cpu()', number=4, setup="from __main__ import cpu")
gpu_time = timeit.timeit('gpu()', number=4, setup="from __main__ import gpu")
print('Time (s) to prform a style tranfer for the style and content images shown. Sum of 4 runs')
print('CPU (s): {:.4f}'.format(cpu_time))
print('GPU (s): {:.4f}'.format(gpu_time))
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

show_n([content_images[content_id], style_images[style_id], stylized_image_cpu, stylized_image_gpu],
       titles=['Original content image', 'Style image', 'Stylized image cpu', 'Stylized image gpu'])

As we can see, the performance is definitely improved. While this varies considerably from run to run, it can be expected an speedup of 10 to 20 times by using the GPU vs the CPU.

This speedup rate also depends on which models are being implemented. Convolutional neural networks, which are tyipically used for images benefit greatly from using the GPU, but fully connected nets or RNNs usually do not increase substantially their performance when using the GPU. 

**IMPORTANT: there are limits to the usage of Google Colab's GPUs. If you intend on running large scale machine learning models, please check Google's conditions first.**